In [11]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the datetime module from the datetime library.
from datetime import datetime

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [12]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [13]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [14]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

644

In [15]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [21]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Cloudiness": city_clouds,
                          "Country": city_country,
                          "Date": city_date,
                          "Humidity": city_humidity,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Wind Speed": city_wind})                                        

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | cape town
Processing Record 2 of Set 1 | saint george
Processing Record 3 of Set 1 | muravlenko
Processing Record 4 of Set 1 | jamestown
Processing Record 5 of Set 1 | qaanaaq
Processing Record 6 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 7 of Set 1 | outjo
Processing Record 8 of Set 1 | benguela
Processing Record 9 of Set 1 | clarksburg
Processing Record 10 of Set 1 | panama city
Processing Record 11 of Set 1 | santa rosa de lima
Processing Record 12 of Set 1 | georgetown
Processing Record 13 of Set 1 | ushuaia
Processing Record 14 of Set 1 | luderitz
Processing Record 15 of Set 1 | nanjing
Processing Record 16 of Set 1 | uhlove
Processing Record 17 of Set 1 | matagami
Processing Record 18 of Set 1 | chalchihuites
Processing Record 19 of Set 1 | vaini
Processing Record 20 of Set 1 | belaya gora
Processing Record 21 of Set 1 | cabo san lucas
Processing Record 22 of 

In [22]:
len(city_data)

603

In [23]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Cape Town,100,ZA,2021-05-26 21:11:06,81,-33.9258,18.4232,57.11,1.74
1,Saint George,1,US,2021-05-26 21:10:25,11,37.1041,-113.5841,91.87,5.99
2,Muravlenko,100,RU,2021-05-26 21:11:38,42,63.7898,74.5230,42.04,22.44
3,Jamestown,90,US,2021-05-26 21:11:39,86,42.0970,-79.2353,72.09,7.00
4,Qaanaaq,35,GL,2021-05-26 21:11:39,78,77.4840,-69.3632,35.62,3.62
5,Outjo,0,NA,2021-05-26 21:11:39,38,-20.1167,16.1500,57.81,7.90
6,Benguela,3,AO,2021-05-26 21:11:40,74,-12.5763,13.4055,75.43,3.29
7,Clarksburg,75,US,2021-05-26 21:11:40,45,39.2807,-80.3445,86.11,11.25
8,Panama City,75,PA,2021-05-26 21:08:39,79,8.9936,-79.5197,87.06,2.64
9,Santa Rosa De Lima,85,SV,2021-05-26 21:11:40,27,13.6247,-87.8936,101.43,7.27


In [29]:
# Reorder the columns
new_column_order = ["City","Country","Date","Lat","Lng","Humidity","Cloudiness","Wind Speed"]

city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Humidity,Cloudiness,Wind Speed
0,Cape Town,ZA,2021-05-26 21:11:06,-33.9258,18.4232,81,100,1.74
1,Saint George,US,2021-05-26 21:10:25,37.1041,-113.5841,11,1,5.99
2,Muravlenko,RU,2021-05-26 21:11:38,63.7898,74.5230,42,100,22.44
3,Jamestown,US,2021-05-26 21:11:39,42.0970,-79.2353,86,90,7.00
4,Qaanaaq,GL,2021-05-26 21:11:39,77.4840,-69.3632,78,35,3.62
5,Outjo,NA,2021-05-26 21:11:39,-20.1167,16.1500,38,0,7.90
6,Benguela,AO,2021-05-26 21:11:40,-12.5763,13.4055,74,3,3.29
7,Clarksburg,US,2021-05-26 21:11:40,39.2807,-80.3445,45,75,11.25
8,Panama City,PA,2021-05-26 21:08:39,8.9936,-79.5197,79,75,2.64
9,Santa Rosa De Lima,SV,2021-05-26 21:11:40,13.6247,-87.8936,27,85,7.27


In [30]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")